# Installation Faker & import dataset

In [325]:
!pip install Faker


In [326]:
from faker import Faker
fake = Faker()

In [327]:
import pandas as pd

# Import dataset ---
url = f"https://docs.google.com/spreadsheets/d/1PIMO76csrwWa7eRCZHQziJQvqGMy7M_EpZLblZjRFMU/gviz/tq?tqx=out:csv&sheet=dataset_clean"
df = pd.read_csv(url)
df

,age,attrition,deplacements_pro,service,trajet_quotidien_en_miles,num_niveau_detude,domaine_detude,identifiant_employe,num_satisfaction_environnement,genre,...,statut,categorie_salaire,nb_entreprises,pourcentage_augmentation,niveau_carriere,temps_de_formation,anciennete,anciennete_poste,derniere_promotion,annees_avec_manager
0,41,Oui,Rare,Ventes,1,2,Sciences,1,2,Femme,...,2,5744 - 9860,6 +,11 - 12,06 - 10,0,06 - 10,03 - 05,0 - 02,03 - 05
1,49,Non,Fréquent,R&D,8,1,Sciences,2,3,Homme,...,1,4230 - 5743,0 - 1,21 +,06 - 10,01 - 03,06 - 10,06 - 10,0 - 02,06 - 10
2,37,Oui,Rare,R&D,2,2,Autre,4,4,Homme,...,1,0 - 2696,6 +,15 - 20,06 - 10,01 - 03,0 - 02,0 - 02,0 - 02,0 - 02
3,33,Non,Fréquent,R&D,3,4,Sciences,5,4,Femme,...,1,2697 - 4229,0 - 1,11 - 12,06 - 10,01 - 03,06 - 10,06 - 10,03 - 05,0 - 02
4,27,Non,Rare,R&D,2,1,Médecine,7,1,Homme,...,1,2697 - 4229,6 +,11 - 12,06 - 10,01 - 03,0 - 02,0 - 02,0 - 02,0 - 02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,Non,Fréquent,R&D,23,2,Médecine,2061,3,Homme,...,1,0 - 2696,0 - 1,15 - 20,10 - 20,01 - 03,03 - 05,0 - 02,0 - 02,03 - 05
1466,39,Non,Rare,R&D,6,1,Médecine,2062,4,Homme,...,2,9861 +,0 - 1,15 - 20,06 - 10,04 +,06 - 10,06 - 10,0 - 02,06 - 10
1467,27,Non,Rare,R&D,4,3,Sciences,2064,2,Homme,...,2,5744 - 9860,0 - 1,15 - 20,06 - 10,0,06 - 10,0 - 02,0 - 02,03 - 05
1468,49,Non,Fréquent,Ventes,2,3,Médecine,2065,4,Homme,...,2,4230 - 5743,0 - 1,13 - 14,10 - 20,01 - 03,06 - 10,06 - 10,0 - 02,06 - 10


# Génération des noms et des prénoms et assignation aux personnes

In [328]:
import numpy as np

In [329]:
# Séparation des lignes entre hommes et
df_f = df[df["genre"] == "Femme"].copy().reset_index()
df_h = df[df["genre"] == "Homme"].copy().reset_index()

# Generation des prénoms féminins et masculines
f_names_list = [fake.first_name_female() for i in range(len(df_f))]
m_names_list = [fake.first_name_male() for i in range(len(df_h))]

# Generation des noms de famille
last_names_list = [fake.last_name() for i in range(len(df))]

# Assignation d'un prénom et d'un nom aléatoires aux femmes
df_f["prenom"] = np.random.choice(f_names_list, size=len(df_f), replace=True)
df_f["nom"] = np.random.choice(last_names_list, size=len(df_f), replace=True)

# Assignation d'un prénom et d'un nom aléatoires aux hommes
df_h["prenom"] = np.random.choice(m_names_list, size=len(df_h), replace=True)
df_h["nom"] = np.random.choice(last_names_list, size=len(df_h), replace=True)

# Regroupement des dataframes df_f et df_h dans le dataframe initial df
df.loc[df_f["index"], ["prenom", "nom"]] = df_f[["prenom", "nom"]].values
df.loc[df_h["index"], ["prenom", "nom"]] = df_h[["prenom", "nom"]].values

# Génération des emails

In [333]:
email_list=[]
def generate_email(prenom, nom, test_doublon):
    p = prenom.strip().lower()[0]
    n = nom.strip().lower().replace(" ","")
    base = f"{p}.{n}@ibm-data.com"
    email = base
    counter = 1
    while email in test_doublon:
        email = f"{p}.{n}{counter}@ibm-data.com"
        counter += 1
    test_doublon.add(email)
    return email

existing_emails = set()
df["email"] = df.apply(lambda row: generate_email(row["prenom"], row["nom"], existing_emails), axis=1)

# Check des doublons prenoms+noms & emails

In [331]:
# détection valeurs dupliquées
df_prenom_nom=df[["prenom","nom"]]
df_prenom_nom

,prenom,nom
0,Rose,Atkins
1,Timothy,Davis
2,Harold,Newman
3,Michelle,Olson
4,Richard,Riley
...,...,...
1465,Ricky,Johnson
1466,Carlos,Holmes
1467,Alejandro,Mclaughlin
1468,Roger,Green


In [332]:
df_prenom_nom.duplicated().sum()

np.int64(16)

In [340]:
df_email=df[["email"]]
df_email

,email
0,r.atkins@ibm-data.com
1,t.davis@ibm-data.com
2,h.newman@ibm-data.com
3,m.olson@ibm-data.com
4,r.riley@ibm-data.com
...,...
1465,r.johnson1@ibm-data.com
1466,c.holmes@ibm-data.com
1467,a.mclaughlin@ibm-data.com
1468,r.green@ibm-data.com


In [341]:
df_email.duplicated().sum()

np.int64(0)

# Sauvegarde dans le dataset

In [335]:
from google.colab import auth
auth.authenticate_user()

In [336]:
import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

In [337]:
sh = gc.open_by_url("https://docs.google.com/spreadsheets/d/1PIMO76csrwWa7eRCZHQziJQvqGMy7M_EpZLblZjRFMU/edit")

In [338]:
worksheet = sh.worksheet("data")

In [339]:
from gspread_dataframe import set_with_dataframe

set_with_dataframe(worksheet, df)